In [1]:
cd ..

/home/agustin/Dropbox/Doctorado/NeuroData/PyLeech


In [2]:
cd ..

/home/agustin/Dropbox/Doctorado/NeuroData


In [3]:
%matplotlib qt5

In [4]:
import os

import PyLeech.Utils.NLDUtils as NLD
import PyLeech.Utils.AbfExtension as abfe

import PyLeech.Utils.CrawlingDatabaseUtils as CDU
import PyLeech.Utils.burstStorerLoader as bStorerLoader
import PyLeech.Utils.burstUtils as burstUtils
import numpy as np
import scipy.signal as spsig
import matplotlib.pyplot as plt
import PyLeech.Utils.filterUtils as filterUtils
import PyLeech.Utils.correlationUtils as corrUtils
import gc 
from itertools import combinations

cdd = CDU.loadDataDict()
print(list(cdd))

['RegistrosDP_PP\\18n05010.pklspikes', 'RegistrosDP_PP\\2018_10_11_0002.pklspikes', 'RegistrosDP_PP\\2018_12_03_0005.pklspikes', 'RegistrosDP_PP\\2018_12_04_0003_0004_0005_0006b.pklspikes', 'RegistrosDP_PP\\2018_12_06_0005.pklspikes', 'RegistrosDP_PP\\2018_12_13_0015.pklspikes', 'RegistrosDP_PP\\NS_DP_PP_0.pklspikes', 'RegistrosDP_PP\\NS_T_DP_PP_0_cut.pklspikes', 'RegistrosDP_PP\\NS_T_DP_PP_1.pklspikes', 'RegistrosDP_PP\\2014_09_25_0008.pklspikes', 'RegistrosDP_PP\\2019_01_28_0001.pklspikes', 'RegistrosDP_PP\\2018_11_06_0004.pklspikes', 'RegistrosDP_PP\\2018_12_03_0000_0001.pklspikes', 'RegistrosDP_PP\\14217000.pklspikes', 'RegistrosDP_PP\\2017_12_08_0002.pklspikes', 'RegistrosDP_PP\\cont10.pklspikes', 'RegistrosDP_PP\\2018_12_13_0001.pklspikes', 'RegistrosDP_PP\\2019_07_22_0002.pklspikes', 'RegistrosDP_PP\\2019_07_22_0009.pklspikes', 'RegistrosDP_PP\\2019_07_22_0011.pklspikes', 'RegistrosDP_PP\\2019_07_23_0004.pklspikes', 'RegistrosDP_PP\\2019_07_23_0008.pklspikes', 'RegistrosDP_PP\\2

In [12]:
a = [print(i, fn) for i, fn in zip(range(len(cdd)),list(cdd))]

0 RegistrosDP_PP\18n05010.pklspikes
1 RegistrosDP_PP\2018_10_11_0002.pklspikes
2 RegistrosDP_PP\2018_12_03_0005.pklspikes
3 RegistrosDP_PP\2018_12_04_0003_0004_0005_0006b.pklspikes
4 RegistrosDP_PP\2018_12_06_0005.pklspikes
5 RegistrosDP_PP\2018_12_13_0015.pklspikes
6 RegistrosDP_PP\NS_DP_PP_0.pklspikes
7 RegistrosDP_PP\NS_T_DP_PP_0_cut.pklspikes
8 RegistrosDP_PP\NS_T_DP_PP_1.pklspikes
9 RegistrosDP_PP\2014_09_25_0008.pklspikes
10 RegistrosDP_PP\2019_01_28_0001.pklspikes
11 RegistrosDP_PP\2018_11_06_0004.pklspikes
12 RegistrosDP_PP\2018_12_03_0000_0001.pklspikes
13 RegistrosDP_PP\14217000.pklspikes
14 RegistrosDP_PP\2017_12_08_0002.pklspikes
15 RegistrosDP_PP\cont10.pklspikes
16 RegistrosDP_PP\2018_12_13_0001.pklspikes
17 RegistrosDP_PP\2019_07_22_0002.pklspikes
18 RegistrosDP_PP\2019_07_22_0009.pklspikes
19 RegistrosDP_PP\2019_07_22_0011.pklspikes
20 RegistrosDP_PP\2019_07_23_0004.pklspikes
21 RegistrosDP_PP\2019_07_23_0008.pklspikes
22 RegistrosDP_PP\2019_07_23_0014.pklspikes
23 Regi

In [7]:
fn = [fn for fn in list(cdd) if '2019_08_30_0006' in fn][0]
print(cdd[fn]['channels'])
ch_dict = {ch: info for ch, info in cdd[fn]['channels'].items() if len(info)>=2}
print(ch_dict)

{'Vm1': 'NS', 'Vm2': '-', 'IN4': 'DP', 'IN5': 'PP or AA', 'IN6': 'PP or AA', 'IN7': 'MA'}
{'Vm1': 'NS', 'IN4': 'DP', 'IN5': 'PP or AA', 'IN6': 'PP or AA', 'IN7': 'MA'}


In [10]:
fn = fn.replace("\\", '/')

arr_dict, time_vector, fs = abfe.getArraysFromAbfFiles(fn, list(ch_dict))
time_step = .1
corr_step = int(10/time_step)
for key in arr_dict.keys():
    arr_dict[key] = corrUtils.processContinuousSignal(arr_dict[key], dt_step=1/fs, kernel_sigma=.5)
    arr_dict[key] = arr_dict[key][int(.1*arr_dict[key].shape[0]):int(.9*arr_dict[key].shape[0])]
    arr_dict[key] = arr_dict[key][::int(time_step * fs)]

time_vector = np.arange(0, arr_dict[key].shape[0]*time_step, time_step)

2527

In [71]:
centers = np.arange(0, NS.shape[0], corr_step, dtype=int) + int(corr_step/2)

for k, j in combinations(range(arr_dict.keys(), 2):
    dcorr_list = []
    for i in np.arange(0, NS.shape[0], corr_step, dtype=int):
        dcorr_list.append(corrUtils.getDistanceCorrelation(arr_dict[k][i:i+corr_step], elems[j][i:i+corr_step]))
    fig, ax = plt.subplots(3, 1, sharex=True)
    ax[0].plot(time_vector, arr_dict   [k])
    ax[1].plot(time_vector, arr_dict[j])
    ax[2].scatter(time_vector[centers[:-1]], dcorr_list[:-1])
    fig.suptitle(ch_dict[k] + '-' + ch_dict[j] + ' correlation')


In [59]:
# fig, ax = plt.subplots(3, 1, sharex=True)
# ax[0].plot(time_vector, NS)
# ax[1].plot(time_vector, n0)
# ax[2].scatter(time_vector[centers[:-1]], dcorr_list[:-1])